<a href="https://colab.research.google.com/github/maxhormazabal/depencendy_parsing/blob/main/p2_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Preprocessing

**Students:** Maximiliano Hormazábal - Mutaz Abueisheh

## Installation of dependencies

In [ ]:
!pip install conllu

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Connecting to Google Drive to read the `.py` file that contains the functions to use.

In [ ]:
# Getting access to Google Drive files
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Import our own functions (they are in a .py file on Google Drive)
import os
os.chdir("/content/drive/MyDrive/MASTER")
from nlu_preprocessing_utils import *
from conllu import parse
import tensorflow as tf
import pandas as pd
import numpy as np
import math
import io

## Reading datasource from its origin on github

In [ ]:
# English
# 'https://raw.githubusercontent.com/UniversalDependencies/UD_English-EWT/master/en_ewt-ud-train.conllu'
# https://raw.githubusercontent.com/UniversalDependencies/UD_English-ParTUT/master/en_partut-ud-train.conllu

base_url = 'https://raw.githubusercontent.com/UniversalDependencies/UD_English-ParTUT/master/'
file_basename = 'en_partut-ud'
(en_train,en_test,en_val) = readConlluDataset(base_url,file_basename)
en_upo2number, en_number2upo, en_nupos = getUposList(en_train)
number2action,action2number = getActionDict()

152194/152194 [==============================] - 0s 0us/step


In [ ]:
!rm nlu_data/original_test.conllu
generateConlluForTesting()

Original file generated in nlu_data/original_test_line.conllu


## Transforming the data source into the initial dataset

In [ ]:
train_df = conlluToDatasetForDependency(en_train,en_upo2number)
test_df = conlluToDatasetForDependency(en_test,en_upo2number)
val_df = conlluToDatasetForDependency(en_val,en_upo2number)

### Checking Projective Arcs

In [ ]:
train_df = train_df.iloc[projectiveArcs(train_df)]
test_df = test_df.iloc[projectiveArcs(test_df)]
val_df = val_df.iloc[projectiveArcs(val_df)]

## Training the Tokenizer

In [ ]:
from keras.preprocessing.text import Tokenizer

text = "root"

for sentence in train_df['form']:
  for word in sentence:
    text = text + " " + word

tokenizer = Tokenizer(oov_token="<OOV>",filters="") 
tokenizer.fit_on_texts([text])
word_index = tokenizer.word_index

#save
with open('nlu_data/tokenizer.json', 'w') as outfile:
    outfile.write(tokenizer.to_json())

## Post-Oracle datasets

In [ ]:
(2,3),(3,3),(5,5),(6,6),(8,8),(10,10),(12,12)

In [ ]:
stack_len = 12
buffer_len = 12
(x_train,action_train,deprel_train) = transformByOracle(train_df,stack_len,buffer_len,en_nupos)
(x_test,action_test,deprel_test) = transformByOracle(test_df,stack_len,buffer_len,en_nupos)
(x_val,action_val,deprel_val) = transformByOracle(val_df,stack_len,buffer_len,en_nupos)

In [ ]:
deprel_train

array(['None', 'None', 'None', ..., 'punct', 'None', 'None'], dtype='<U21')

## X-Variables Tokenization

In [ ]:
x_train_token = applyTokenizer(x_train,stack_len,buffer_len,tokenizer)
x_test_token = applyTokenizer(x_test,stack_len,buffer_len,tokenizer)
x_val_token = applyTokenizer(x_val,stack_len,buffer_len,tokenizer)

## Y-Variables encoding

In [ ]:
number2deprel,deprel2number = getDeprelDict(deprel_train)

deprel_train,number2deprel_train,deprel2number_train = deprelToNumericalByTrain(deprel_train,number2deprel,deprel2number)
deprel_test,number2deprel_test,deprel2number_test = deprelToNumericalByTrain(deprel_test,number2deprel,deprel2number)
deprel_val,number2deprel_val,deprel2number_val = deprelToNumericalByTrain(deprel_val,number2deprel,deprel2number)

In [ ]:
action_encod_train = tf.keras.utils.to_categorical(action_train)
deprel_encod_train = tf.keras.utils.to_categorical(deprel_train)
action_encod_test = tf.keras.utils.to_categorical(action_test)
deprel_encod_test = tf.keras.utils.to_categorical(deprel_test)
action_encod_val = tf.keras.utils.to_categorical(action_val)
deprel_encod_val = tf.keras.utils.to_categorical(deprel_val)

### Making lengths of deprel equal between sets. Using the maximum value of deprels

In [ ]:
max_len = max([deprel_encod_train.shape[1],deprel_encod_test.shape[1],deprel_encod_val.shape[1]])

deprel_encod_train = tf.keras.utils.pad_sequences(deprel_encod_train,maxlen=max_len,padding='post')
deprel_encod_test = tf.keras.utils.pad_sequences(deprel_encod_test,maxlen=max_len,padding='post')
deprel_encod_val = tf.keras.utils.pad_sequences(deprel_encod_val,maxlen=max_len,padding='post')

Creating folder to save the data depending of the size of stack and buffer

In [ ]:
folder_name = str(stack_len)+"stack"+str(buffer_len)+"buffer"
path = "nlu_data/"+folder_name
!mkdir -p {path}

In [ ]:
# function to save the numpy array as a file and re-use it avoiding preprocessing steps
# It will be save in your current directory (seted in "os.chdir("/content/drive/MyDrive/MASTER")")

# Saving train data
np.save(path+'/x_train.npy', x_train_token) 
np.save(path+'/action_train.npy', action_encod_train)
np.save(path+'/deprel_train.npy', deprel_encod_train)

# Saving test data
np.save(path+'/x_test.npy', x_test_token) 
np.save(path+'/action_test.npy', action_encod_test)
np.save(path+'/deprel_test.npy', deprel_encod_test)

# Saving val data
np.save(path+'/x_val.npy', x_val_token) 
np.save(path+'/action_val.npy', action_encod_val)
np.save(path+'/deprel_val.npy', deprel_encod_val)

# With the following you can read the file and create the numpy array again
# new_x_data = np.load('x_data.npy',allow_pickle=True)
# new_action_data = np.load('action_data.npy',allow_pickle=True)